# Preprocessing Flux Distributions for Objective II #

In [3]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.plotting import multiplot_envelopes,plot_envelope,plot_pecaso_dfba,\
                              multi_two_stage_char_contours,two_stage_char_contour
from mcpecaso.core import mcPECASO
import mcpecaso as mcpecaso
from mcpecaso.core.substrate_dependent_envelopes import linear_uptake, logistic_uptake
import time


mcpecaso.settings.time_end = 100
mcpecaso.settings.initial_biomass = 0.05
mcpecaso.settings.initial_substrate = 500
mcpecaso.settings.num_timepoints = 10000
mcpecaso.settings.scope = 'extrema'

pd.options.display.max_rows = 100

num_cores = multiprocessing.cpu_count()

In [2]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"
import os

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


In [3]:
pd.options.display.max_rows=600
pd.options.display.max_columns=150
wt_model = cameo.models.bigg.iJO1366

relevant_reactions = [reaction.id for reaction in wt_model.reactions 
                      if reaction.subsystem not in ['Transport, Outer Membrane',
                                                    'Transport, Inner Membrane',
                                                    'Inorganic Ion Transport and Metabolism',
                                                    'Transport, Outer Membrane Porin',
                                                    'Intracellular demand',
                                                    'Extracellular exchange']]
relevant_reactions_set = set(relevant_reactions)

metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxypropanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxyacetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthioribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

model = cameo.models.bigg.iJO1366
model.solver = 'gurobi'

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}


aa_metabolism_subsystems=['Alanine and Aspartate Metabolism',
                          'Arginine and Proline Metabolism',
                          'Cysteine Metabolism',
                          'Glutamate Metabolism',
                          'Glycine and Serine Metabolism',
                          'Histidine Metabolism',
                          'Methionine Metabolism',
                          'Threonine and Lysine Metabolism',
                          'Tyrosine, Tryptophan, and Phenylalanine Metabolism',
                          'Valine, Leucine, and Isoleucine Metabolism']
ijo_sub = {reaction.id:reaction.subsystem if reaction.subsystem not in aa_metabolism_subsystems
                                                    else 'Amino Acid Metabolism'
           for reaction in model.reactions}


Academic license - for non-commercial use only


In [4]:
pickling_off = open("flux_dicts.pickle","rb")
flux_dicts = pickle.load(pickling_off)
pickling_off.close()

In [5]:
flux_dict_prod = flux_dicts['productivity']
flux_dict_prod_75_yield = flux_dicts['productivity_75_yield']
flux_dict_yield_2_prod = flux_dicts['yield_2_productivity']

In [6]:
def reduced_fva(flux_dict, product, used_reactions, unused_reactions, template_model, slack=0):
    
    model = deepcopy(template_model)
    
    biomass_flux, substrate_flux, target_flux = flux_dict[product]
    target_rxn = [reaction for reaction in model.exchanges 
                  if reaction.reactants[0].name==metabolite_lookup_dict[product]][0]
    
    with model:
        for reaction in list(unused_reactions):
                if model.reactions.get_by_id(reaction).reversibility:
                    model.reactions.get_by_id(reaction).bounds = (-slack, slack)
                else:
                    model.reactions.get_by_id(reaction).bounds = (0, slack)

         #CONSTRAINTS
        model.reactions.EX_glc__D_e.bounds = (substrate_flux-slack, 1000)
        model.reactions.get_by_id(target_rxn.id).bounds = (target_flux-slack, target_flux+slack)

        mut_df = cameo.flux_variability_analysis(model,fraction_of_optimum=1,reactions=used_reactions).data_frame
        mut_pfba = cameo.pfba(model).data_frame.reindex(used_reactions)

    mut_pfba[mut_pfba.abs()<0.001]=0
    mut_df[mut_df.abs()<0.001]=0
    temp = pd.concat([mut_pfba,mut_df], axis=1, sort=True)
    temp.columns = ['pfba','lb','ub']
    temp['range'] = temp.ub-temp.lb
    temp[temp.abs()<0.001]=0
    
    return {product:temp}

In [7]:
def global_fva_parser(fva_dict, flex_cutoff=0.001):
    parsed_fva = {}
    temp_dict = deepcopy(fva_dict)
    for product in temp_dict.keys():
        temp_dict[product] = temp_dict[product].fillna(0)
        pfba_values = temp_dict[product][(temp_dict[product].range<=flex_cutoff)].pfba.values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'lb'] = pfba_values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'ub'] = pfba_values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'range'] = 0
    
    
    products = list(temp_dict.keys())

    range_df = pd.concat([temp_dict[product].range for product in temp_dict.keys()],axis=1)
    range_df = range_df.fillna(0)
    flexible_reactions_set = set(range_df[range_df.any(axis=1)].index)
    inflexible_reactions_set = relevant_reactions_set - flexible_reactions_set
    
    
    pfba_df = pd.concat([temp_dict[product].pfba for product in temp_dict.keys()],axis=1)
    pfba_df = pfba_df.fillna(0)
    pfba_df.columns = products
    
    used_reactions_set = set(pfba_df[pfba_df.any(axis=1)].index)
    unused_reactions_set = relevant_reactions_set - used_reactions_set

    parsed_fva['used_reactions_set'] = used_reactions_set
    parsed_fva['unused_reactions_set'] = unused_reactions_set
    parsed_fva['flexible_reactions_set'] = flexible_reactions_set
    parsed_fva['inflexible_reactions_set'] = inflexible_reactions_set
    parsed_fva['inflexible'] = {}
    parsed_fva['inflexible']['pfba'] = pd.concat([temp_dict[product].reindex(inflexible_reactions_set.intersection(used_reactions_set)).pfba for product in temp_dict.keys()],axis=1)
    parsed_fva['inflexible']['pfba'].columns = products
    parsed_fva['flexible'] = {}
    parsed_fva['flexible']['pfba'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set)).pfba for product in temp_dict.keys()],axis=1)
    parsed_fva['flexible']['pfba'].columns = products
    columns = []
    for product in temp_dict.keys():
        columns.append((product,'lb'))
        columns.append((product,'ub'))
    columns = pd.MultiIndex.from_tuples(columns)
    parsed_fva['flexible']['bounds'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set))[['lb','ub']] for product in temp_dict.keys()], axis=1)
    parsed_fva['flexible']['bounds'].columns = columns
    parsed_fva['flexible']['range'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set)).range for product in temp_dict.keys()],axis=1)
    parsed_fva['flexible']['range'].columns = products
    
    #products flexible, products inflexible but active, products inflexible AND inactive
    flex_products = []
    flex_products_no = []
    inflex_used_products = []
    inflex_used_products_no = []
    inflex_unused_products = []
    inflex_unused_products_no = []
    for reaction in parsed_fva['flexible']['pfba'].index:
        flex_products.append(set(parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]!=0].index))
        flex_products_no.append(len(parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]!=0].index))
        inflex_products = parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]==0].index
        inflex_used_products.append(set(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)!=0].index))
        inflex_used_products_no.append(len(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)!=0].index))
        inflex_unused_products.append(set(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)==0].index))
        inflex_unused_products_no.append(len(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)==0].index))

    parsed_fva['flexible']['products'] = pd.DataFrame(data={'flex':flex_products, 'flex_no':flex_products_no, 'inflex_used':inflex_used_products,
                                                 'inflex_used_no':inflex_used_products_no, 'inflex_unused':inflex_unused_products,
                                                 'inflex_unused_no':inflex_unused_products_no}, index=parsed_fva['flexible']['pfba'].index)

    
    sorted_order = parsed_fva['flexible']['products'].sort_values(by=['flex_no','inflex_used_no'],ascending=[False,False]).index
    parsed_fva['flexible']['pfba'] = parsed_fva['flexible']['pfba'].reindex(sorted_order)
    parsed_fva['flexible']['bounds'] = parsed_fva['flexible']['bounds'].reindex(sorted_order) 
    parsed_fva['flexible']['range'] = parsed_fva['flexible']['range'].reindex(sorted_order)
    parsed_fva['flexible']['products'] = parsed_fva['flexible']['products'].reindex(sorted_order) 
    
    
    return temp_dict,parsed_fva

# FIRST PASS FVA #

In [34]:
#FIRST PASS FVA - NO EXTRA CONSTRAINTS APART FROM WILDTYPE. FVA DONE ONLY ON RELEVANT REACTIO

start_time = time.time()

result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_prod_75_yield, product, relevant_reactions_set,
                                                                         [], model, slack=0) 
                                                     for product in flux_dict_prod_75_yield.keys())


fva = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print(time.time()-start_time)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  3.0min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  3.0min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  3.2min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  3.2min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  3.3min
[Parallel(

794.7344982624054


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed: 13.2min finished


In [35]:
pickling_on = open("fva_prod_75_yield.pickle","wb")
pickle.dump(fva, pickling_on)
pickling_on.close() 

In [36]:
pickle_off = open("fva_prod_75_yield.pickle","rb")
fva = pickle.load(pickle_off)
pickle_off.close()

In [74]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva,parsed_fva = global_fva_parser(fva,flex_cutoff=0.001)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set'].intersection(parsed_fva['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set'].intersection(parsed_fva['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva['flexible_reactions_set'].intersection(parsed_fva['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva['flexible_reactions_set'].intersection(parsed_fva['used_reactions_set'])))

unused_reactions_set_1 = parsed_fva['unused_reactions_set']
used_reactions_set_1 = parsed_fva['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1191
Unused Inflexible Reactions:  973
Used Inflexible Reactions:  218 

Flexible Reactions:  302
Unused Flexible Reactions:  137
Used Flexible Reactions:  165


In [66]:
parsed_fva['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
NDPK1,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
SUCDi,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
TRSARr,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
G3PAT161,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
SUCOAS,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
R15BPK,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
NADH18pp,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
DHORD2,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
VALTA,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
PPK,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0


In [39]:
wt_model.solver = 'gurobi'
wt_df = cameo.flux_variability_analysis(wt_model,fraction_of_optimum=1,reactions=relevant_reactions).data_frame
wt_pfba = cameo.pfba(wt_model).data_frame.reindex(relevant_reactions)

#Pruning out small values. If a flux is less than 0.001, make it equal to 0
wt_pfba[wt_pfba.abs()<0.001]=0
wt_df[wt_df.abs()<0.001]=0

#Consolidate pfba and fva into a single dataframe
comparison_df = pd.concat([wt_pfba,wt_df], axis=1, sort=True)
comparison_df.columns = ['pfba','lb','ub']

#Add a column describing the range of allowable fluxes for each reaction
comparison_df['range'] = comparison_df.ub-comparison_df.lb
wt_fva = comparison_df
wt_used_reactions = set(list(wt_fva[wt_fva.pfba!=0].index))

# SECOND PASS FVA #

In [40]:
#SECOND PASS FVA - REACTIONS UNUSED FOR ALL PRODUCTS FROM FIRST PASS CONSTRAINED TO ZERO
start_time = time.time()

result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_prod_75_yield, product, used_reactions=used_reactions_set_1,
                                                                          unused_reactions=unused_reactions_set_1,
                                                                          template_model=model, slack=0.001) for product in flux_dict_prod_75_yield.keys())


fva_second = {list(result_dict.keys())[0]: list(result_dict.values())[0]
                                for result_dict in result_list}

print("Completed in ", time.time()-start_time, " s")

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   33.1s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   39.7s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   41.8s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   45.3s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   48.7s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   53.5s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:   57.2s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  1.7min
[Parallel(

Completed in  408.9949440956116  s


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  6.8min finished


In [41]:
pickling_on = open("fva_second_prod_75_yield.pickle","wb")
pickle.dump(fva_second, pickling_on)
pickling_on.close() 

In [4]:
pickle_off = open("fva_second_prod_75_yield.pickle","rb")
fva_second = pickle.load(pickle_off)
pickle_off.close()

In [8]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_second,parsed_fva_2 = global_fva_parser(fva_second,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set'].intersection(parsed_fva_2['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set'].intersection(parsed_fva_2['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set'].intersection(parsed_fva_2['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set'].intersection(parsed_fva_2['used_reactions_set'])))

unused_reactions_set_2 = parsed_fva_2['unused_reactions_set']
used_reactions_set_2 = parsed_fva_2['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1409
Unused Inflexible Reactions:  1110
Used Inflexible Reactions:  299 

Flexible Reactions:  84
Unused Flexible Reactions:  0
Used Flexible Reactions:  84


In [9]:
parsed_fva_2['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
FBA,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
NADH18pp,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
PRPPS,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
FRD3,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
R15BPK,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
DHORDfum,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
SUCDi,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
ADK1,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
DHORD2,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0
TALA,"{(S)-Propanediol, O-Acetyl-L-serine, Thymine, ...",70,{},0,{},0


# THIRD PASS FVA #

In [48]:
#THIRD PASS FVA - REACTIONS UNUSED FOR ALL PRODUCTS FROM SECOND PASS CONSTRAINED TO ZERO
start_time = time.time()

result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_prod_75_yield, product, used_reactions=used_reactions_set_2,
                                                                         unused_reactions=unused_reactions_set_2,
                                                                          template_model=model, slack=0.001) for product in flux_dict_prod_75_yield.keys())


fva_third = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print("Completed in ", time.time()-start_time, " s")

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   50.5s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   53.3s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   54.0s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   57.7s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  1.8min
[Parallel(

Completed in  394.68140411376953  s


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  6.6min finished


In [49]:
pickling_on = open("fva_third.pickle","wb")
pickle.dump(fva_third, pickling_on)
pickling_on.close()

In [10]:
pickle_off = open("fva_third.pickle","rb")
fva_third = pickle.load(pickle_off)
pickle_off.close()

In [11]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_third_temp,parsed_fva_3 = global_fva_parser(fva_third,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set'].intersection(parsed_fva_3['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set'].intersection(parsed_fva_3['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set'].intersection(parsed_fva_3['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set'].intersection(parsed_fva_3['used_reactions_set'])))

unused_reactions_set_3 = parsed_fva_3['unused_reactions_set']
used_reactions_set_3 = parsed_fva_3['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1409
Unused Inflexible Reactions:  1110
Used Inflexible Reactions:  299 

Flexible Reactions:  84
Unused Flexible Reactions:  0
Used Flexible Reactions:  84


# FOURTH PASS FVA #

In [4]:
parsed_fva_3_temp = deepcopy(parsed_fva_3)
fva_third_temp = deepcopy(fva_third)


NameError: name 'parsed_fva_3' is not defined

In [1]:
#FOURTH PASS FVA - FLEXIBLE REACTIONS THAT CAN BE ZERO FOR ALL PRODUCTS REMOVED ONE BY ONE AND FVA UPDATED AFTER EACH REMOVAL
parsed_fva_3_temp['flexible']['products'] = parsed_fva_3_temp['flexible']['products'].sort_index().sort_values(by=['flex_no','inflex_used_no'], ascending=[False,False])


old_list = ['DHORD2', 'SUCDi', 'NADH17pp', 'R1PK', 'FRD2', 'FBA', 'R15BPK', 'TALA',
           'ADK1', 'NADH16pp', 'PRPPS', 'PPM', 'DHORDfum', 'FBA3', 'PFK', 'PFK_3',
           'HEX1', 'XYLI2', 'HEX7', 'GART', 'FTHFLi', 'THRD', 'THRA', 'RNDR3',
           'RNDR2', 'NDPK6', 'NDPK5', 'FLDR2', 'RNDR4', 'NDPK7', 'FADRx2', 'AMPN',
           'NTD7', 'TRPS1', 'IDOND', 'IDOND2', 'SERD_L', 'FBP']
new_list = [reaction for reaction in old_list if reaction in deletable_reactions]
new_list.extend(set(deletable_reactions)-set(new_list))

deletable_reactions = parsed_fva_3_temp['flexible']['products'].reindex(new_list).sort_values(by=['flex_no','inflex_used_no'],ascending=[False,False]).index

for reaction in reversed(deletable_reactions):
    if (reaction in parsed_fva_3_temp['flexible']['products'].index) and (reaction not in wt_used_reactions):
        zeroflag=1
        flex_products = [metabolite_lookup_dict[product] for product in parsed_fva_3_temp['flexible']['products']['flex'].loc[reaction]]
        for product in flex_products:
            if not((parsed_fva_3_temp['flexible']['bounds'].loc[reaction].loc[metabolite_names_dict[product]].lb<=0) and (parsed_fva_3_temp['flexible']['bounds'].loc[reaction].loc[metabolite_names_dict[product]].ub>=0)):
                zeroflag=0
        if zeroflag:
            print("Removing reaction: ", reaction)
            used_reactions_set = parsed_fva_3_temp['used_reactions_set'] - set([reaction])
            unused_reactions_set = parsed_fva_3_temp['unused_reactions_set'].union(set([reaction]))
            for i,product in enumerate(flex_products):
                print('Evaluating Product ', i+1, ' of ', len(flex_products))
                temp = reduced_fva(flux_dict_prod_75_yield, metabolite_names_dict[product],
                                                                             unused_reactions=unused_reactions_set,
                                                                             used_reactions=used_reactions_set,
                                                                             template_model=model, slack=0.001)
                fva_third_temp[metabolite_names_dict[product]] = list(temp.values())[0]
            
            fva_third_temp,parsed_fva_3_temp = global_fva_parser(fva_third_temp,flex_cutoff=0.008)


NameError: name 'parsed_fva_3_temp' is not defined

In [130]:
fva_fourth = deepcopy(fva_third_temp)


In [139]:
pickling_on = open("fva_fourth_prod_75_yield.pickle","wb")
pickle.dump(fva_fourth, pickling_on)
pickling_on.close() 

In [140]:
pickle_off = open("fva_fourth_prod_75_yield.pickle","rb")
fva_fourth = pickle.load(pickle_off)
pickle_off.close()

In [141]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_fourth,parsed_fva_4 = global_fva_parser(fva_fourth,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set'].intersection(parsed_fva_4['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set'].intersection(parsed_fva_4['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set'].intersection(parsed_fva_4['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set'].intersection(parsed_fva_4['used_reactions_set'])))

unused_reactions_set_4 = parsed_fva_4['unused_reactions_set']
used_reactions_set_4 = parsed_fva_4['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1431
Unused Inflexible Reactions:  1119
Used Inflexible Reactions:  312 

Flexible Reactions:  62
Unused Flexible Reactions:  0
Used Flexible Reactions:  62


In [142]:
parsed_fva_4['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
FRD3,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
FBA,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
SUCDi,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
NADH16pp,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
TALA,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
NADH18pp,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",70,{},0,{},0
PFK_3,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",69,{},0,{Dihydroxyacetone},1
FBA3,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",69,{},0,{Dihydroxyacetone},1
PFK,"{5-Methylthioribose, Succinate, Thymine, (R)-G...",69,{},0,{Dihydroxyacetone},1
SERD_L,"{(R)-Glycerate, Glycolate, Urea, α-Ketoglutara...",5,{},0,"{5-Methylthioribose, Succinate, Thymine, O-Ace...",65


# Fifth Pass FVA - Product Specific Deletions#

In [143]:
# Identifies product specific constraints that are possible:
# From fourth pass FVA, perform following steps:
# 1. Identify which products still have flexible reactions and iterate through each:
# 2. For each flexible reaction in that product:
# 3. Count products whose PFBA result falls within the flux range for that reaction
# 4. Also, identify the most frequently occurring flux value and the number of times it appears.
# 5. Arrange in descending order of number of times the most frequently occurring flux value occurs.

complete_products_set = set(fva_fourth.keys())
flex_products_lists = list(parsed_fva_4['flexible']['products']['flex'].values)
flex_products_sets = set([frozenset(product_list) for product_list in flex_products_lists])
temp_list = []
for temp_set in flex_products_sets:
    temp_list.extend(list(temp_set))
flex_products = (set(temp_list))
temp_dict = {}
for i,product in enumerate(flex_products):
    print('Analyzing Product :', product)
    reactions = parsed_fva_4['flexible']['range'][product][parsed_fva_4['flexible']['range'][product]!=0].index
    products_within_bounds = []
    products_within_bounds_no = []
    mode_pfba = []
    mode_no =[]
    for reaction in reactions:
        ub = parsed_fva_4['flexible']['bounds'].loc[reaction][product]['ub']
        lb = parsed_fva_4['flexible']['bounds'].loc[reaction][product]['lb']
        pfba = parsed_fva_4['flexible']['pfba'].loc[reaction][complete_products_set-set([product])]
        temp_df = pfba[(pfba>=lb)&(pfba<=ub)]
        temp_df = temp_df.round(decimals=3)
        products_within_bounds.append(list(temp_df.index))
        products_within_bounds_no.append(len(temp_df.index))
        if len(temp_df.index):
            mode_pfba.append(temp_df.mode().values[0])
            mode_no.append(len(temp_df[temp_df==temp_df.mode().values[0]].index))
        else:
            mode_pfba.append(np.nan)
            mode_no.append(0)
    data = {'products_within_bounds':products_within_bounds,'number':products_within_bounds_no,'mode_pfba':mode_pfba,'mode_no':mode_no}
    df = pd.DataFrame(data,index=reactions)
    temp_df = pd.concat([parsed_fva_4['flexible']['bounds'].loc[reactions][product],parsed_fva_4['flexible']['pfba'].loc[reactions][product]],axis=1)
    product_df = pd.concat([temp_df,df],axis=1)
    product_df = product_df.sort_values(by='mode_no',ascending=False)
    product_df.columns = ['lower','upper','pfba','products_within_bounds','number_within_bounds','mode','no_mode']
    temp_dict[product]=product_df
    display(product_df)

Analyzing Product : 5-Methylthioribose


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,10.469530,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
NADH18pp,0.000000,10.469219,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
TALA,-1.140673,1.092835,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",68,0.000,46
PFK_3,0.000000,2.233508,1.092835,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,21
FBA3,0.000000,2.233508,1.092835,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,21
PFK,0.000000,2.233508,1.140673,"[D-Gluconic Acid, L-Arginine, L-Homoserine, D-...",38,0.000,3
FBA,0.000000,2.233508,1.140673,"[D-Gluconic Acid, L-Arginine, L-Homoserine, D-...",37,0.000,2
NADH16pp,0.000000,10.469219,10.469219,"[Succinate, Glycine, Uridine, D-Gluconic Acid,...",50,4.427,2
SUCDi,1.022617,11.494146,1.022818,"[Uridine, Indole, Guanine, Xanthosine, Adenosi...",29,1.031,1


Analyzing Product : Succinate


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
TALA,-4.177407,0.032175,0.000000,"[Glycine, Allantoin, Uridine, D-Gluconic Acid,...",66,0.000,46
PFK_3,0.000000,4.209582,0.032175,"[Glycine, Allantoin, Uridine, D-Gluconic Acid,...",69,0.000,21
FBA3,0.000000,4.209582,0.032175,"[Glycine, Allantoin, Uridine, D-Gluconic Acid,...",69,0.000,21
SUCDi,0.000000,7.167460,0.000000,"[Glycine, Allantoin, Uridine, D-Gluconic Acid,...",69,0.000,17
PFK,0.000000,4.209582,4.177407,"[Glycine, Allantoin, Uridine, D-Gluconic Acid,...",69,0.000,3
FBA,0.000000,4.209582,4.177407,"[Glycine, Allantoin, Uridine, D-Gluconic Acid,...",68,0.000,2
NADH16pp,0.000000,7.167460,7.166460,"[D-Gluconic Acid, D-Lactate, Acetate, 5-Ketogl...",18,4.427,2
FRD3,2.643346,9.811022,2.644228,[],0,NaN,0
NADH18pp,2.643012,9.810688,2.644228,[],0,NaN,0


Analyzing Product : Thymine


...

,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,10.266554,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
NADH18pp,0.000000,10.265432,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
TALA,-2.351096,0.996521,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",68,0.000,46
PFK_3,0.000000,3.347618,0.996521,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,21
FBA3,0.000000,3.347618,0.996521,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,21
PFK,0.000000,3.347618,2.351096,"[Allantoin, Uridine, D-Gluconic Acid, L-Argini...",59,0.000,3
FBA,0.000000,3.347618,2.351096,"[Allantoin, Uridine, D-Gluconic Acid, L-Argini...",58,0.000,2
NADH16pp,0.000000,10.265432,10.265432,"[Succinate, Glycine, Uridine, D-Gluconic Acid,...",49,4.427,2
SUCDi,0.658877,10.927431,0.658877,"[Uridine, D-Gluconic Acid, L-Homoserine, Indol...",36,0.681,1


Analyzing Product : Cytidine


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,9.698080,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
NADH18pp,0.000000,9.697768,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
PFK_3,0.000000,2.644993,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,20
FBA3,0.000000,2.644993,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,20
PFK,0.000000,2.644993,2.644993,"[Uridine, D-Gluconic Acid, L-Arginine, L-Homos...",49,0.000,3
FBA,0.000000,2.644993,2.644993,"[Uridine, D-Gluconic Acid, L-Arginine, L-Homos...",48,0.000,2
NADH16pp,0.000000,9.697768,9.697768,"[Succinate, Glycine, Uridine, D-Gluconic Acid,...",42,4.427,2
SUCDi,1.370431,11.070511,1.370633,"[Indole, Guanine, Glycerol3-phosphate, (R)-Pro...",12,1.417,1
TALA,-2.832613,-0.187620,-0.187620,"[Acetate, Xanthosine, Glycolate, (R)-Glycerate...",7,-1.035,1


Analyzing Product : Formate


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,8.131593,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
NADH18pp,0.000000,8.131482,0.000000,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",69,0.000,68
PFK_3,0.000000,0.186884,0.186884,"[Succinate, Glycine, Uridine, Acetate, Xanthos...",26,0.000,21
FBA3,0.000000,0.186884,0.186884,"[Succinate, Glycine, Uridine, Acetate, Xanthos...",26,0.000,21
TRDR,0.040159,0.055476,0.054476,"[Succinate, Glycine, Allantoin, Uridine, D-Glu...",50,0.041,15
NADH16pp,0.000000,8.131482,8.130482,"[Succinate, Glycine, D-Gluconic Acid, D-Lactat...",27,4.427,2
PFK,0.000000,0.186884,0.000000,"[Dihydroxyacetone, L-Cysteine]",2,0.000,2
FBA,0.000000,0.186884,0.000000,[L-Cysteine],1,0.000,1
SUCDi,2.053784,10.187377,2.053784,"[Glycerol3-phosphate, (R)-Propanediol, Glycola...",5,2.459,1
TALA,3.679401,3.866285,3.679401,[],0,NaN,0


In [144]:
# Deepcopy parsed_fva and fva to perform product specific constrained fva
temp_fva = deepcopy(fva_fourth)
temp_parsed = deepcopy(parsed_fva_4)

In [145]:
deletable_reactions = {}
for product in metabolite_names_dict.values():
    temp = temp_dict[product][temp_dict[product]['mode']==0].index
    for reaction in temp:
        if (parsed_fva_4['flexible']['bounds'][product].loc[reaction]['lb']<=0) and (parsed_fva_4['flexible']['bounds'][product].loc[reaction]['ub']>=0):
            deletable_reactions[reaction] = deletable_reactions.get(reaction,[])+[product]
df_dict = {'products':[products for products in deletable_reactions.values()],
           'no':[len(parsed_fva_4['flexible']['pfba'].loc[reaction][parsed_fva_4['flexible']['pfba'].loc[reaction].round(decimals=3)==0]) for reaction in deletable_reactions.keys()]}
deletable_reactions = pd.DataFrame(data=df_dict,index=deletable_reactions.keys())
deletable_reactions = deletable_reactions.sort_values(by='no',ascending=False)

In [146]:
deletable_reactions

,products,no
FRD3,"[Spermidine, L-Tryptophan, Thymidine, L-Phenyl...",69
NADH18pp,"[Spermidine, L-Tryptophan, Thymidine, L-Phenyl...",69
FDH4pp,[(R)-Propanediol],68
FTHFD,[(R)-Propanediol],68
MALS,"[L-Idonic Acid, 5-Ketogluconate, (R)-Propanediol]",66
TRPS1,[α-Ketoglutarate],65
ICL,"[L-Idonic Acid, 5-Ketogluconate, (R)-Propanediol]",64
THD2pp,[Dihydroxyacetone],50
TALA,"[Spermidine, L-Tryptophan, Thymidine, L-Phenyl...",47
AKGDH,[5-Ketogluconate],21


In [147]:
# FIFTH PASS FVA -  IMPLEMENT PRODUCT SPECIFIC DELETIONS OR ZERO CONSTRAINTS
# 1. For each flexible product:
# 2. Filter reactions that have '0' as their most frequently occurring flux value
# 3. For each such reaction:
# 4. if 0 is in the flux range for that product:
# 5. Perform FVA with new constraints for that specific product
# 6. Update fva, parsed_fva, perform the 
# 7. Continue
prod_specific_deletions = {}
for i,product in enumerate(metabolite_names_dict.values()):
    print('Evaluating product ', i+1, ' of 70')
    for reaction in deletable_reactions.index:
        if reaction in temp_parsed['flexible']['products'].index:
            if ((temp_parsed['flexible']['bounds'][product].loc[reaction]['lb']<=0) and (temp_parsed['flexible']['bounds'][product].loc[reaction]['ub']>=0) and temp_parsed['flexible']['range'][product].loc[reaction]>0):
                print('Removing reaction: ',reaction)
                prod_specific_deletions[product]=prod_specific_deletions.get(product,[]) + [reaction]
                temp = reduced_fva(flux_dict_prod_75_yield, product, unused_reactions=temp_parsed['unused_reactions_set'].union(set(prod_specific_deletions.get(product))),
                                    used_reactions=temp_parsed['used_reactions_set'], template_model=model, slack=0.001)
                temp_fva[product]=temp[product]
                temp_fva,temp_parsed = global_fva_parser(temp_fva,flex_cutoff=0.008)



Evaluating product  1  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpwt8f0lqr.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpdpv44oni.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  2  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpbvqos1xn.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp187vz9i_.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  3  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpz4ge99ht.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP forma

Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  22  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp43sfabrp.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpj7_hmpq_.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  23  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp4vac84zr.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpq92zenmo.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  24  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpdzn4b0rd.lp
Reading time = 0.03 seconds
: 1805 

Evaluating product  43  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp8qf6sqpb.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpfk15o3dy.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  44  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpgwsrj4j5.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp4yajue9o.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  45  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpvr9fxdps.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP fo

Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpejvmudfb.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp28moe5f9.lp
Reading time = 0.04 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  65  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp01ietaxn.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmphbpzf4ia.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  66  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp8qtsh213.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\

In [149]:
fva_fifth = deepcopy(temp_fva)

In [150]:
pickling_on = open("fva_fifth_prod_75_yield.pickle","wb")
pickle.dump(fva_fifth, pickling_on)
pickling_on.close() 

In [151]:
pickle_off = open("fva_fifth_prod_75_yield.pickle","rb")
fva_fifth = pickle.load(pickle_off)
pickle_off.close()

In [152]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_fifth,parsed_fva_5 = global_fva_parser(fva_fifth,flex_cutoff=0.015)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set'].intersection(parsed_fva_5['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set'].intersection(parsed_fva_5['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set'].intersection(parsed_fva_5['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set'].intersection(parsed_fva_5['used_reactions_set'])))


Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1491
Unused Inflexible Reactions:  1119
Used Inflexible Reactions:  372 

Flexible Reactions:  2
Unused Flexible Reactions:  0
Used Flexible Reactions:  2


In [156]:
min_biomass = min(parsed_fva_5['inflexible']['pfba'].loc['BIOMASS_Ec_iJO1366_core_53p95M'])
for product in fva_fifth.keys():
    cutoff = 0.001*parsed_fva_5['inflexible']['pfba'][product].loc['BIOMASS_Ec_iJO1366_core_53p95M']/min_biomass
    fva_fifth[product]['pfba'][abs(fva_fifth[product]['pfba'])<=cutoff]=0
    fva_fifth[product]['lb'][abs(fva_fifth[product]['lb'])<=cutoff]=0
    fva_fifth[product]['ub'][abs(fva_fifth[product]['ub'])<=cutoff]=0
    fva_fifth[product]['range'] = fva_fifth[product]['ub'] - fva_fifth[product]['lb']
    fva_fifth[product]['range'][abs(fva_fifth[product]['range'])<=cutoff]=0

In [169]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_final,parsed_fva_final = global_fva_parser(fva_fifth,flex_cutoff=0.03)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set'].intersection(parsed_fva_final['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set'].intersection(parsed_fva_final['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set'].intersection(parsed_fva_final['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set'].intersection(parsed_fva_final['used_reactions_set'])))


Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1493
Unused Inflexible Reactions:  1122
Used Inflexible Reactions:  371 

Flexible Reactions:  0
Unused Flexible Reactions:  0
Used Flexible Reactions:  0


In [170]:
pickling_on = open("fva_final_prod_75_yield.pickle","wb")
pickle.dump(fva_final, pickling_on)
pickling_on.close() 

In [8]:
pickle_off = open("fva_final_prod_75_yield.pickle","rb")
fva_final = pickle.load(pickle_off)
pickle_off.close()

In [172]:
pickling_on = open("parsed_fva_final_prod_75_yield.pickle","wb")
pickle.dump(parsed_fva_final, pickling_on)
pickling_on.close() 

In [9]:
pickle_off = open("parsed_fva_final_prod_75_yield.pickle","rb")
parsed_fva_final = pickle.load(pickle_off)
pickle_off.close()

In [174]:
parsed_fva_final['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no


In [175]:
len(parsed_fva_final['used_reactions_set'])

371

In [12]:
len(fva_final.keys())

70